In [ ]:
import pyspark
pyspark.__version__

'3.5.1'

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('EDA').getOrCreate()

25/09/27 13:15:06 WARN Utils: Your hostname, Pongsakorns-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.1.34 instead (on interface en0)
25/09/27 13:15:06 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/27 13:15:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark

In [14]:
df_pyspark = spark.read.csv("sales_data_set.csv", header=True, inferSchema=True)
print(f"{'Type:'.ljust(20)} {type(df_pyspark)}")
print(f"{'Total rows:'.ljust(20)} {df_pyspark.count()}")
print(f"{'Columns:'.ljust(20)} {df_pyspark.columns}")

Type:                <class 'pyspark.sql.dataframe.DataFrame'>
Total rows:          421570
Columns:             ['Store', 'Dept', 'Date', 'Weekly_Sales', 'IsHoliday']


In [8]:
df_pyspark.show(5)

+-----+----+----------+------------+---------+
|Store|Dept|      Date|Weekly_Sales|IsHoliday|
+-----+----+----------+------------+---------+
|    1|   1|05/02/2010|     24924.5|    false|
|    1|   1|12/02/2010|    46039.49|     true|
|    1|   1|19/02/2010|    41595.55|    false|
|    1|   1|26/02/2010|    19403.54|    false|
|    1|   1|05/03/2010|     21827.9|    false|
+-----+----+----------+------------+---------+
only showing top 5 rows



In [ ]:
df_pyspark.show(5)